# NYC Apartment Search



## Setup

In [35]:
import json
import pathlib
import urllib.parse

import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db

#display all columns.
pd.set_option('display.max_columns', None)

from sqlalchemy.orm import declarative_base

In [36]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
DATA_311_FILE = DATA_DIR / "resource" / "erm2-nwe9.geojson"
DATA_TREE_FILE = DATA_DIR / "resource" / "5rq2-4hqu.geojson"
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "JkEVszrAdzm7qGLzLjvQVBnnM"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "FILL_ME_IN"
DB_USER = "FILL_ME_IN"
DB_URL = f"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [37]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing
The process of cleaning & filtering the data includes:
1. Removing unnecessary columns, and only keeping columns needed to answer questions;
2. Remove invalid data points;
3. Normalize column names & column types where needed;
4. Normalize the Spatial Reference Identifiers (SRID) of any geometry

In [38]:
def download_nyc_geojson_data(url, force=False):
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    
    filename = DATA_DIR / url_path
    
    if force or not filename.exists():
        filename.parent.mkdir(parents=True, exist_ok=True)
        print(f"Downloading {url} to {filename}...")
        
        # Download data using requests
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Save the downloaded data as a GeoJSON file
            with open(filename, "w") as f:
                json.dump(response.json(), f)
            
            print(f"Done downloading {url}.")
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")

    else:
        print(f"Reading from {filename}...")

    return filename

In [39]:
def download_and_clean_311_data(download = True):
    '''
    https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9
    
    The complete data has the following columns:
    ['location_state', 'facility_type', 'intersection_street_2', 'city',
    'location_zip', 'park_borough', 'latitude', 'road_ramp', 'created_date',
    'agency', 'park_facility_name', 'location_address', 'agency_name',
    'descriptor', 'bbl', 'location_city', 'open_data_channel_type',
    'cross_street_2', 'bridge_highway_direction', 'longitude',
    'bridge_highway_segment', 'street_name', 'incident_address',
    'address_type', 'incident_zip', 'unique_key', 'complaint_type',
    'y_coordinate_state_plane', 'status', 'bridge_highway_name',
    'location_type', 'due_date', 'taxi_company_borough',
    'taxi_pick_up_location', 'x_coordinate_state_plane',
    'resolution_description', 'community_board',
    'resolution_action_updated_date', 'intersection_street_1',
    'closed_date', 'vehicle_type', 'cross_street_1', 'borough', 'landmark',
    'location']
       
    I only downloaded `created_date`, `descriptor`, `incident_zip`, `location` column from the database 
    where = created_date > '2022-01-01T00:00:00'.
    '''    
    # URL for getting the JSON file
    url = "https://data.cityofnewyork.us/resource/" + NYC_DATA_311+ "?$$app_token="+ NYC_DATA_APP_TOKEN + \
    "&$select= created_date, descriptor, incident_zip AS zipcode, location &$where=created_date>'2022-01-01T00:00:00' \
    &$limit=1000"
    
    # download and save the data
    if download: 
        download_nyc_geojson_data(url, force=True)
    
    # load the data.
    geodf_311_data = gpd.read_file(DATA_311_FILE)
    
    # drop rows where there is no zipcode
    geodf_311_data = geodf_311_data.dropna(subset=['zipcode'])
    
    #return the data object.
    return geodf_311_data

In [40]:
def download_and_clean_tree_data(download = True):
    '''
    https://dev.socrata.com/foundry/data.cityofnewyork.us/5rq2-4hqu
    
    The complete data has the following columns:
    ['nta', 'health', 'zipcode', 'latitude', 'nta_name', 'state',
    'trnk_wire', 'y_sp', 'brnch_othe', 'root_grate', 'tree_id', 'steward',
    'spc_common', 'trnk_other', 'x_sp', 'brnch_ligh', 'problems',
    'longitude', 'boro_ct', 'zip_city', 'spc_latin', 'stump_diam',
    'boroname', 'st_senate', 'user_type', 'status', 'brnch_shoe',
    'curb_loc', 'cncldist', 'guards', 'st_assem', 'cb_num', 'address',
    'sidewalk', 'root_other', 'created_at', 'borocode', 'block_id',
    'trnk_light', 'tree_dbh', 'root_stone', 'the_geom']
       
    I only downloaded `zipcode` and `the_geom` column from the database.
    '''
    # URL for getting the JSON file
    url = "https://data.cityofnewyork.us/resource/" + NYC_DATA_TREES+ "?$$app_token="+ NYC_DATA_APP_TOKEN + \
    "&$select= zipcode,the_geom&$limit=1000"
    # download and save the data
    if download:
        download_nyc_geojson_data(url, force=True)
    
    # load and the data.
    geodf_tree_data = gpd.read_file(DATA_TREE_FILE)
    
    # drop rows where there is no zipcode
    geodf_tree_data = geodf_tree_data.dropna(subset=['zipcode'])

    #return the data object.
    return geodf_tree_data 

In [41]:
def load_and_clean_zipcodes(zipcode_datafile):
    '''
    `zipcode_datafile` has following columns:
    ['ZIPCODE', 'BLDGZIP', 'PO_NAME', 'POPULATION', 'AREA', 'STATE', 'COUNTY', 
     'ST_FIPS', 'CTY_FIPS', 'URL', 'SHAPE_AREA', 'SHAPE_LEN', 'geometry']    
    '''
    
    geodf_zipcode_data = gpd.read_file(zipcode_datafile)
    
    column_names = geodf_zipcode_data.columns
    # change all column names to lowercase.
    geodf_zipcode_data.columns = [element.lower() for element in column_names]
    
    # drop unnecessary columns
    geodf_zipcode_data = geodf_zipcode_data.drop(columns = ["bldgzip","shape_area","shape_len","url"])
    
    return geodf_zipcode_data

In [42]:
def load_and_clean_zillow_data(zillow_datafile):
    df_zillow_data = pd.read_csv(zillow_datafile)
    
    # rename `RegionName` as `zipcode`
    df_zillow_data.rename(columns={'RegionName': 'zipcode'}, inplace=True)
    
    # drop unnecessary columns
    df_zillow_data = df_zillow_data.drop(columns = ["RegionID","SizeRank","RegionType","StateName"])
    
    return df_zillow_data

In [43]:
# If download == True, the program will download the data from the internet and cover the old data.
# For the first time runing this program, turn it True.
def load_all_data(download = True):
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data(download)
    geodf_tree_data = download_and_clean_tree_data(download)
    df_zillow_data = load_and_clean_zillow_data(ZILLOW_DATA_FILE)
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [44]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data(True)

Done downloading https://data.cityofnewyork.us/resource/erm2-nwe9.geojson?$$app_token=JkEVszrAdzm7qGLzLjvQVBnnM&$select= created_date, descriptor, incident_zip AS zipcode, location &$where=created_date>'2022-01-01T00:00:00'     &$limit=1000.
Done downloading https://data.cityofnewyork.us/resource/5rq2-4hqu.geojson?$$app_token=JkEVszrAdzm7qGLzLjvQVBnnM&$select= zipcode,the_geom&$limit=1000.


In [45]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   zipcode     263 non-null    object  
 1   po_name     263 non-null    object  
 2   population  263 non-null    float64 
 3   area        263 non-null    float64 
 4   state       263 non-null    object  
 5   county      263 non-null    object  
 6   st_fips     263 non-null    object  
 7   cty_fips    263 non-null    object  
 8   geometry    263 non-null    geometry
dtypes: float64(2), geometry(1), object(6)
memory usage: 18.6+ KB


In [46]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

,zipcode,po_name,population,area,state,county,st_fips,cty_fips,geometry
0,11436,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,"POLYGON ((1038098.252 188138.380, 1038141.936 ..."
1,11213,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,"POLYGON ((1001613.713 186926.440, 1002314.243 ..."
2,11212,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,"POLYGON ((1011174.276 183696.338, 1011373.584 ..."
3,11225,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,"POLYGON ((995908.365 183617.613, 996522.848 18..."
4,11218,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,"POLYGON ((991997.113 176307.496, 992042.798 17..."


In [47]:
geodf_311_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 961 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   created_date  961 non-null    datetime64[ns]
 1   descriptor    955 non-null    object        
 2   zipcode       961 non-null    object        
 3   geometry      905 non-null    geometry      
dtypes: datetime64[ns](1), geometry(1), object(2)
memory usage: 37.5+ KB


In [48]:
geodf_311_data.head()

,created_date,descriptor,zipcode,geometry
0,2023-11-28 12:00:00,Derelict Vehicles,10033,POINT (-73.93298 40.85212)
1,2023-11-28 12:00:00,Derelict Vehicles,11430,POINT (-73.76696 40.65347)
2,2023-11-28 02:04:10,Graffiti,11204,POINT (-73.99161 40.61774)
3,2023-11-28 00:44:00,Street Light Out,10033,POINT (-73.93104 40.85051)
4,2023-11-28 00:33:00,Street Light Cycling,10305,POINT (-74.06476 40.60801)


In [49]:
geodf_tree_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   zipcode   1000 non-null   object  
 1   geometry  1000 non-null   geometry
dtypes: geometry(1), object(1)
memory usage: 15.8+ KB


In [50]:
geodf_tree_data.head()

,zipcode,geometry
0,11375,POINT (-73.84422 40.72309)
1,11357,POINT (-73.81868 40.79411)
2,11211,POINT (-73.93661 40.71758)
3,11211,POINT (-73.93446 40.71354)
4,11215,POINT (-73.97598 40.66678)


In [51]:
df_zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6722 entries, 0 to 6721
Columns: 110 entries, zipcode to 2023-09-30
dtypes: float64(105), int64(1), object(4)
memory usage: 5.6+ MB


In [52]:
df_zillow_data.head()

,zipcode,State,City,Metro,CountyName,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
0,77494,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1606.206406,1612.779844,1622.201575,1630.392427,1632.411500,1636.206864,1644.894632,1643.390762,1636.971443,1620.756709,1613.330035,1595.875303,1587.956334,1571.722899,1580.226011,1577.360511,1575.019249,1570.437252,1560.570981,1559.473860,1550.038637,1532.579831,1510.242213,1486.489733,1507.802059,1533.972404,1560.775400,1560.921517,1569.898260,1578.146822,1578.913743,1567.921730,1577.355514,1590.264504,1606.499320,1604.421318,1611.899807,1618.293318,1621.761808,1617.780728,1614.801931,1615.532634,1611.448067,1601.793861,1590.638710,1590.488945,1594.393947,1598.275260,1604.520456,1610.434841,1621.770680,1625.351916,1626.773326,1625.936662,1624.890088,1633.134397,1625.910897,1629.721928,1626.974655,1636.612419,1640.770934,1647.007094,1651.109647,1657.091010,1649.138584,1645.635867,1637.868714,1649.266605,1662.133844,1665.130638,1674.053593,1679.239150,1688.863037,1690.829750,1692.870693,1713.964591,1744.244031,1792.325270,1827.718052,1856.436987,1872.532636,1884.990347,1910.473392,1917.549543,1930.611286,1926.224851,1936.273593,1950.552503,1975.971322,1999.610405,2023.300973,2024.379276,2018.999370,2006.725802,1990.684558,1994.653463,2027.438438,2042.237444,2049.325559,2016.531345,2023.438976,2031.558202,2046.144009,2053.486247,2055.771355
1,77449,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1257.814660,1255.268025,1262.170452,1274.955754,1285.526052,1295.665673,1296.650395,1300.868081,1301.898486,1302.881427,1299.693120,1296.038652,1288.469114,1287.887883,1296.983247,1310.096909,1316.314107,1308.568281,1300.912474,1295.072347,1292.669100,1288.731946,1279.219942,1270.826144,1270.159851,1280.804521,1290.962297,1297.787179,1299.429207,1305.319483,1313.028808,1314.471319,1318.936586,1318.360343,1325.904220,1327.061006,1338.242460,1342.947774,1346.339152,1347.977879,1353.781015,1360.934194,1361.964965,1362.711392,1358.947439,1356.220524,1356.988632,1353.259235,1350.062265,1348.472443,1354.870417,1359.648555,1367.652345,1372.060549,1380.044029,1377.192680,1378.572673,1375.046953,1372.524410,1373.553574,1381.286565,1390.962381,1402.096689,1398.546295,1390.275169,1390.301160,1403.523253,1422.490142,1432.232355,1431.127611,1437.327132,1448.187834,1458.760896,1465.848888,1471.032905,1490.807492,1527.079852,1565.896231,1605.220723,1614.231294,1619.765081,1631.133342,1636.650862,1656.262010,1650.745900,1677.114958,1697.240420,1720.421368,1698.930423,1697.851631,1735.015389,1802.088342,1802.928430,1790.160022,1752.950850,1749.697900,1738.217986,1747.305840,1758.407295,1758.891075,1762.980879,1771.751591,1779.338402,1795.384582,1799.631140
2,77084,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1258.992453

## Part 2: Storing Data

In [ ]:
!createdb 4501_project_database

In [ ]:
def setup_new_postgis_database(username, db_name):
    raise NotImplementedError()

In [ ]:
setup_new_postgis_database(DB_USER, DB_NAME)

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
engine = db.create_engine(DB_URL)

#### Option 1: SQL

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables
ZIPCODE_SCHEMA = """
TODO
"""

NYC_311_SCHEMA = """
TODO
"""

NYC_TREE_SCHEMA = """
TODO
"""

ZILLOW_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [ ]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
with engine.connect() as connection:
    pass

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

class Tree(Base):
    __tablename__ = "trees"

    ...


In [ ]:
Base.metadata.create_all(engine)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [ ]:
def write_dataframes_to_table(tablename_to_dataframe):
    # write INSERT statements or use pandas/geopandas to write SQL
    raise NotImplemented()

In [ ]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [ ]:
write_dataframes_to_table(tablename_to_dataframe)

#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)